In [1]:
from bootcamp.databricks_compat import display, dbutils, spark
from data.config import DATA_DIR

In [2]:
# Drop table if you need to start fresh
spark.sql("DROP TABLE IF EXISTS nytaxi_yellow_2")

DataFrame[]

In [3]:
%fs ls /databricks-datasets/nyctaxi/tripdata/yellow/

path,name,size,modificationTime
dbfs:/home/jovyan/work/data/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_sample.tsv.gz,yellow_tripdata_sample.tsv.gz,81887950,1765285617688


In [4]:
# Read 3 files (~3M rows total)
urls = [
    "s3a://datasets-documentation/nyc-taxi/trips_0.gz",
    "s3a://datasets-documentation/nyc-taxi/trips_1.gz",
    "s3a://datasets-documentation/nyc-taxi/trips_2.gz"
]

nytaxi_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("sep", "\t") \
    .option("codec", "gzip") \
    .load(urls)

print(f"Total rows: {nytaxi_df.count()}")  # ~30M rows

Total rows: 3000317


In [5]:
nytaxi_df.show()

+----------+---------+-----------+-------------------+------------+-------------------+------------------+------------+------------------+------------------+------------------+------------------+---------------+------------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+------+-------+--------+-------------------+--------------+---------------+-------------+-----------------+-----------------+--------------+--------------------+-----------+--------------------+---------------+----------------+--------------+------------------+------------------+---------------+--------------------+------------+
|   trip_id|vendor_id|pickup_date|    pickup_datetime|dropoff_date|   dropoff_datetime|store_and_fwd_flag|rate_code_id|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|passenger_count|     trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amo

In [6]:
nytaxi_df.write.format("delta").mode("overwrite").saveAsTable("nytaxi_yellow_2")

In [8]:
%%sql

SELECT vendor_id, count(*) as count
FROM nytaxi_yellow_2
WHERE Payment_Type = "CRE" 
GROUP BY vendor_id

vendor_id,count
1,530147
2,605477


In [9]:
%%sql

DESCRIBE DETAIL nytaxi_yellow_2

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures
delta,46114428-fbf1-45c3-888c-0bc1099cd656,spark_catalog.default.nytaxi_yellow_2,None,file:/home/jovyan/work/data/spark-warehouse/nytaxi...,2025-12-09 14:08:11.575000,2025-12-09 14:35:00.002000,[],[],3,158048473,{},1,2,"['appendOnly', 'invariants']"


In [10]:
%fs ls /user/hive/warehouse/nytaxi_yellow_2

path,name,size,modificationTime
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2/.part-00000-a9d0c269-c6b4-4d95-a72f-dd4819bd2fb6-c000.snappy.parquet.crc,.part-00000-a9d0c269-c6b4-4d95-a72f-dd4819bd2fb6-c000.snappy.parquet.crc,412020,1765290899683
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2/.part-00000-aa7a446d-9170-4c43-aa84-8886e80a0ee2-c000.snappy.parquet.crc,.part-00000-aa7a446d-9170-4c43-aa84-8886e80a0ee2-c000.snappy.parquet.crc,412020,1765289306303
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2/.part-00000-d5cf7a3c-ec50-4b68-aa9e-0ecf1eca19c7-c000.snappy.parquet.crc,.part-00000-d5cf7a3c-ec50-4b68-aa9e-0ecf1eca19c7-c000.snappy.parquet.crc,1235128,1765289803788
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2/.part-00001-058d65d3-02db-4101-bccb-5912421b3ffb-c000.snappy.parquet.crc,.part-00001-058d65d3-02db-4101-bccb-5912421b3ffb-c000.snappy.parquet.crc,411440,1765290893812
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2/.part-00001-7d411caa-bf20-498e-b85b-766864d23ea3-c000.snappy.parquet.crc,.part-00001-7d411caa-bf20-498e-b85b-766864d23ea3-c000.snappy.parquet.crc,411440,1765289306002
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2/.part-00002-756cd422-a2d5-4279-a1a2-cbf5c5605fb7-c000.snappy.parquet.crc,.part-00002-756cd422-a2d5-4279-a1a2-cbf5c5605fb7-c000.snappy.parquet.crc,411320,1765289307094
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2/.part-00002-a911f475-2584-48c4-9bc7-b4d7a51a6969-c000.snappy.parquet.crc,.part-00002-a911f475-2584-48c4-9bc7-b4d7a51a6969-c000.snappy.parquet.crc,411320,1765290896844
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2/_delta_log,_delta_log/,0,1765290900016
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2/part-00000-a9d0c269-c6b4-4d95-a72f-dd4819bd2fb6-c000.snappy.parquet,part-00000-a9d0c269-c6b4-4d95-a72f-dd4819bd2fb6-c000.snappy.parquet,52737436,1765290899684
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2/part-00000-aa7a446d-9170-4c43-aa84-8886e80a0ee2-c000.snappy.parquet,part-00000-aa7a446d-9170-4c43-aa84-8886e80a0ee2-c000.snappy.parquet,52737436,1765289306303


In [0]:

%sql

DESCRIBE HISTORY nytaxi_yellow_2

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2023-02-21T17:58:00.000+0000,4143689667304850,futurexskills2023@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1323925794992161),0221-172534-c5915m4y,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 14796313, numOutputBytes -> 733199871)",null,Databricks-Runtime/11.3.x-scala2.12
2,2023-02-21T17:50:09.000+0000,4143689667304850,futurexskills2023@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1323925794992161),0221-172534-c5915m4y,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 14387371, numOutputBytes -> 727614101)",null,Databricks-Runtime/11.3.x-scala2.12
1,2023-02-21T17:44:20.000+0000,4143689667304850,futurexskills2023@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(1323925794992161),0221-172534-c5915m4y,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 13380122, numOutputBytes -> 649386094)",null,Databricks-Runtime/11.3.x-scala2.12
0,2023-02-21T17:36:18.000+0000,4143689667304850,futurexskills2023@gmail.com,CREATE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1323925794992161),0221-172534-c5915m4y,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 14092413, numOutputBytes -> 690359412)",null,Databricks-Runtime/11.3.x-scala2.12


In [0]:

%sql

DESCRIBE DETAIL nytaxi_yellow_2

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,cf653c14-c6ba-4f13-9862-42c1e0070003,spark_catalog.default.nytaxi_yellow_2,null,dbfs:/user/hive/warehouse/nytaxi_yellow_2,2023-02-21T17:32:32.283+0000,2023-02-21T17:58:00.000+0000,List(),4,2800559478,Map(),1,2


In [0]:
%fs

ls /user/hive/warehouse/nytaxi_yellow_2

path,name,size,modificationTime
dbfs:/user/hive/warehouse/nytaxi_yellow_2/_delta_log/,_delta_log/,0,0
dbfs:/user/hive/warehouse/nytaxi_yellow_2/part-00000-0c8fff98-eb45-44d7-912c-cb8ecd0bd727-c000.snappy.parquet,part-00000-0c8fff98-eb45-44d7-912c-cb8ecd0bd727-c000.snappy.parquet,733199871,1677002271000
dbfs:/user/hive/warehouse/nytaxi_yellow_2/part-00000-24bf36bd-c852-4247-a89f-20d26f4d9d84-c000.snappy.parquet,part-00000-24bf36bd-c852-4247-a89f-20d26f4d9d84-c000.snappy.parquet,690359412,1677000967000
dbfs:/user/hive/warehouse/nytaxi_yellow_2/part-00000-4a0d5fe4-82d8-4a74-9890-4a90b8ae5888-c000.snappy.parquet,part-00000-4a0d5fe4-82d8-4a74-9890-4a90b8ae5888-c000.snappy.parquet,727614101,1677001800000
dbfs:/user/hive/warehouse/nytaxi_yellow_2/part-00000-a1583181-ca0b-4e5b-b1ec-765d6da3fec0-c000.snappy.parquet,part-00000-a1583181-ca0b-4e5b-b1ec-765d6da3fec0-c000.snappy.parquet,649386094,1677001452000


In [ ]:
%%sql

SELECT vendor_id, count(*) as count
FROM nytaxi_yellow_2
WHERE Payment_Type = "CRE" 
GROUP BY vendor_id

vendor_id,count
1,530147
2,605477


In [0]:
%sql

SELECT vendor_name, count(*) as count
FROM nytaxi_yellow_2
WHERE Payment_Type = "Credit" 
GROUP BY vendor_name

vendor_name,count
CMT,5954194
VTS,6707165


In [0]:
yellow_df = spark.sql("select * from nytaxi_yellow_2")

In [22]:
yellow_df.write.format("delta").partitionBy("Payment_Type").mode("overwrite").saveAsTable("nytaxi_yellow_2_part")


In [23]:

%%sql

DESCRIBE DETAIL nytaxi_yellow_2_part

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures
delta,9944c958-745e-491d-85c4-41906389b3f1,spark_catalog.default.nytaxi_yellow_2_part,None,file:/home/jovyan/work/data/spark-warehouse/nytaxi...,2025-12-09 13:58:41.348000,2025-12-09 13:58:48.964000,['payment_type'],[],12,160091158,{},1,2,"['appendOnly', 'invariants']"


In [25]:
%fs ls /user/hive/warehouse/nytaxi_yellow_2_part

path,name,size,modificationTime
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2_part/_delta_log,_delta_log/,0,1765288728977
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2_part/payment_type=CRE,payment_type=CRE/,0,1765288724513
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2_part/payment_type=CSH,payment_type=CSH/,0,1765288726279
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2_part/payment_type=DIS,payment_type=DIS/,0,1765288728846
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2_part/payment_type=NOC,payment_type=NOC/,0,1765288728870


In [26]:
%fs ls /user/hive/warehouse/nytaxi_yellow_2_part/Payment_Type=CRE/

path,name,size,modificationTime
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2_part/Payment_Type=CRE/.part-00002-73d42767-1f2b-41e9-a35d-6b6d9e3563e6.c000.snappy.parquet.crc,.part-00002-73d42767-1f2b-41e9-a35d-6b6d9e3563e6.c000.snappy.parquet.crc,157264,1765288726155
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2_part/Payment_Type=CRE/.part-00006-85e69cad-3805-4e76-9fb1-ff68fdca13cb.c000.snappy.parquet.crc,.part-00006-85e69cad-3805-4e76-9fb1-ff68fdca13cb.c000.snappy.parquet.crc,157296,1765288726258
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2_part/Payment_Type=CRE/.part-00010-c2abf170-b913-4ad6-aed1-d540b27d4aa9.c000.snappy.parquet.crc,.part-00010-c2abf170-b913-4ad6-aed1-d540b27d4aa9.c000.snappy.parquet.crc,156968,1765288726265
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2_part/Payment_Type=CRE/part-00002-73d42767-1f2b-41e9-a35d-6b6d9e3563e6.c000.snappy.parquet,part-00002-73d42767-1f2b-41e9-a35d-6b6d9e3563e6.c000.snappy.parquet,20128332,1765288726156
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2_part/Payment_Type=CRE/part-00006-85e69cad-3805-4e76-9fb1-ff68fdca13cb.c000.snappy.parquet,part-00006-85e69cad-3805-4e76-9fb1-ff68fdca13cb.c000.snappy.parquet,20132714,1765288726259
dbfs:/home/jovyan/work/data/spark-warehouse/nytaxi_yellow_2_part/Payment_Type=CRE/part-00010-c2abf170-b913-4ad6-aed1-d540b27d4aa9.c000.snappy.parquet,part-00010-c2abf170-b913-4ad6-aed1-d540b27d4aa9.c000.snappy.parquet,20090810,1765288726265


In [27]:
%%sql

SELECT vendor_id, count(*) as count
FROM nytaxi_yellow_2_part
WHERE Payment_Type = "CRE" 
GROUP BY vendor_id

vendor_id,count
1,530147
2,605477
